# 🎯 Affine Autoencoder Adversarial Attack Analysis

**Minimal interface for comprehensive adversarial attack analysis on Structured 2D+6D Affine Autoencoder**

## **Attack Methods:**
- **FGSM** (Fast Gradient Sign Method) - Single-step gradient attack
- **PGD** (Projected Gradient Descent) - Iterative gradient attack  
- **Latent Space** - Direct manipulation of content vs transform latents

## **Key Features:**
- **Content vs Transform** vulnerability analysis
- **7-row visualization** (original → clean → final → attack → analysis)
- **Attack effectiveness** metrics across multiple epsilon values
- **Method comparison** plots and statistics

In [ ]:
# 📦 IMPORTS & SETUP
import torch
import matplotlib.pyplot as plt
import numpy as np
import random

# Affine autoencoder modules
import affine_autoencoder_shared as shared
import structured_2d6d_autoencoder as s2d6d
from affine_adversarial_attacks import *

print("✅ All imports successful!")
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")

In [ ]:
# ⚙️ CONFIGURATION
CONFIG = {
    # Model parameters
    'content_latent_dim': 2,
    'transform_latent_dim': 6, 
    'total_latent_dim': 8,
    
    # Data parameters
    'batch_size_test': 64,
    'data_dir': '../data',
    
    # Attack parameters
    'n_attack_samples': 8,
    'attack_epsilons': [0.05, 0.1, 0.15, 0.2],
    
    # Model parameters
    'alpha': 1.0,
    'beta': 0.008,
    'mixed_precision': False,
    'device_preference': 'cuda'
}

print("🎯 AFFINE AUTOENCODER ATTACK CONFIGURATION:")
print(f"  Model: {CONFIG['total_latent_dim']}D unified VAE")
print(f"  Attack samples: {CONFIG['n_attack_samples']}")
print(f"  Attack epsilons: {CONFIG['attack_epsilons']}")
print(f"  Beta (KL weight): {CONFIG['beta']}")

In [ ]:
# ?️ SETUP DEVICE & DATA
device = shared.get_cloud_device(CONFIG)
train_loader, test_loader = shared.get_cloud_mnist_loaders(
    batch_size_test=CONFIG['batch_size_test'],
    data_dir=CONFIG['data_dir']
)

print(f"Device: {device}")
print(f"Test loader: {len(test_loader)} batches of {test_loader.batch_size}")

In [ ]:
# 🏗️ LOAD TRAINED MODEL
print("Available models:")
shared.list_saved_models()

# Load your trained affine autoencoder
model_file = "structured_2d6d_simplified_20250721_014252.pth"  # Update this!
model, loaded_config, device = shared.load_model_cloud(model_file)

print(f"\n✅ Loaded model: {model_file}")
print(f"Model architecture: Content({CONFIG['content_latent_dim']}) + Transform({CONFIG['transform_latent_dim']}) = {CONFIG['total_latent_dim']}D")

In [ ]:
# ? COMPREHENSIVE ATTACK ANALYSIS
print("🎯 Starting Comprehensive Affine Autoencoder Attack Analysis...")
print("Testing how attacks affect:")
print("  - Content latent space (digit identity)")  
print("  - Transform latent space (affine parameters)")
print("  - Clean reconstructions (decoder output)")
print("  - Final reconstructions (after affine transform)")
print()

# Run all attacks with analysis
attack_results = run_comprehensive_affine_attacks(model, test_loader, device, CONFIG)

print("\n🏁 Attack analysis completed!")

In [ ]:
# ? COMPARE ATTACK METHODS
print("📊 Comparing attack methods effectiveness...")

# Generate comparison plots
compare_attack_methods(attack_results)

print("✅ Attack comparison completed!")

# 🔍 SINGLE ATTACK DEMO (Optional)
epsilon = 0.1
print(f"🎯 Single Attack Demo: FGSM with ε={epsilon}")

# Get random test samples
test_iter = iter(test_loader)
test_images, test_labels = next(test_iter)
random_indices = random.sample(range(test_images.size(0)), 8)
samples = test_images[random_indices].to(device)
labels = test_labels[random_indices]

# Run FGSM attack
attacker = AffineAdversarialAttacks(model, device)
fgsm_attack_imgs = attacker.fgsm_attack(samples, labels, epsilon)

# Visualize and analyze
visualize_affine_attack_analysis(samples, fgsm_attack_imgs, model, device, 
                               "FGSM Demo", epsilon, labels)

demo_results = analyze_attack_effectiveness(samples, fgsm_attack_imgs, model, device, 
                                          "FGSM Demo", epsilon)

print("✅ Single attack demonstration completed!")

## 🎯 Understanding the Results

### **Visualization Rows:**
1. **Original Images** - Input MNIST digits
2. **Original Clean Recon** - Decoder output (before affine)
3. **Original Final Recon** - After affine transformation
4. **Attack Images** - Perturbed inputs  
5. **Attack Clean Recon** - How attacks affect digit decoding
6. **Attack Final Recon** - How attacks affect final output
7. **Latent Comparison** - Content vs Transform latent changes

### **Key Metrics:**
- **Content Shift**: Attack effect on digit identity (2D latent)
- **Transform Shift**: Attack effect on affine parameters (6D latent)  
- **Content/Transform Ratio**: Which space is more vulnerable
- **Clean vs Final Diff**: Decoder vs affine vulnerability

### **Attack Insights:**
- **High Content/Transform Ratio** → Affects digit identity more
- **Low Content/Transform Ratio** → Affects spatial transform more
- **Latent Space Attack** → Direct latent manipulation
- **FGSM vs PGD** → Single-step vs iterative attacks

### **Expected Behavior:**
- **Content latents** should be more robust (digit identity preservation)
- **Transform latents** may be more vulnerable (spatial parameters)
- **Constrained affine** should limit extreme transformations
- **Unified 8D VAE** should show balanced regularization effects